In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
#load the data 
dtype_spec_test = {
    'column_6_name': 'str',  # Replace 'column_6_name' with the actual name of column 6 in merged_test.csv
}

dtype_spec_train = {
    'column_7_name': 'str',  # Replace 'column_7_name' with the actual name of column 7 in merged_train.csv
}
clean_train = pd.read_csv('merged_train_clean.csv', dtype=dtype_spec_test, low_memory=False)
clean_test = pd.read_csv('merged_test_clean.csv', dtype=dtype_spec_test, low_memory=False)

In [3]:
# Convert the datetime column to a pandas datetime object
clean_train['date'] = pd.to_datetime(clean_train['date'])
clean_test['date'] = pd.to_datetime(clean_test['date'])

# Extract features from the datetime column
clean_train['year'] = clean_train['date'].dt.year
clean_train['month'] = clean_train['date'].dt.month
clean_train['day'] = clean_train['date'].dt.day
clean_train['day_of_week'] = clean_train['date'].dt.dayofweek

clean_test['year'] = clean_test['date'].dt.year
clean_test['month'] = clean_test['date'].dt.month
clean_test['day'] = clean_test['date'].dt.day
clean_test['day_of_week'] = clean_test['date'].dt.dayofweek


In [4]:
# Create a new column that combines 'storetype' and 'promo' in the desired format
clean_train['storetype_promo'] = clean_train.apply(lambda row: f"storetype_{row['storetype']}_haspromo" if row['promo'] == 1 else f"storetype_{row['storetype']}_nopromo", axis=1)
clean_test['storetype_promo'] = clean_test.apply(lambda row: f"storetype_{row['storetype']}_haspromo" if row['promo'] == 1 else f"storetype_{row['storetype']}_nopromo", axis=1)

# Use pd.get_dummies to create dummy variables for the new column
clean_train = pd.get_dummies(clean_train, columns=['storetype_promo'], drop_first=False)
clean_test = pd.get_dummies(clean_test, columns=['storetype_promo'], drop_first=False)

# Display the first few rows to verify
print(clean_train.head())
print(clean_test.head())

   store  dayofweek       date  sales  customers  open  promo stateholiday  \
0      1          5 2015-07-31   5263        555     1      1            0   
1      2          5 2015-07-31   6064        625     1      1            0   
2      3          5 2015-07-31   8314        821     1      1            0   
3      4          5 2015-07-31  13995       1498     1      1            0   
4      5          5 2015-07-31   4822        559     1      1            0   

   schoolholiday storetype  ... day  day_of_week  \
0              1         c  ...  31            4   
1              1         a  ...  31            4   
2              1         a  ...  31            4   
3              1         c  ...  31            4   
4              1         a  ...  31            4   

   storetype_promo_storetype_a_haspromo  storetype_promo_storetype_a_nopromo  \
0                                 False                                False   
1                                  True                   

In [5]:
# Create dummy variables for storetype and assortment
clean_train = pd.get_dummies(clean_train, columns=['storetype', 'assortment'], drop_first=True)
clean_test = pd.get_dummies(clean_test, columns=['storetype', 'assortment'], drop_first=True)


In [6]:
# create dummy variables for promo interval
clean_train = pd.get_dummies(clean_train, columns=['promointerval'], drop_first=True)
clean_test = pd.get_dummies(clean_test, columns=['promointerval'], drop_first=True)

In [7]:
# Convert 'promo2sinceweek' to a date only if both 'promo2sinceweek' and 'promo2sinceyear' are not NaN
clean_train['promo2_start_date'] = clean_train.apply(
    lambda row: pd.to_datetime(f"{int(row['promo2sinceyear'])}{int(row['promo2sinceweek']):02}1", format='%Y%W%w', errors='coerce')
    if pd.notna(row['promo2sinceweek']) and pd.notna(row['promo2sinceyear'])
    else pd.NaT,
    axis=1
)

# Extract the month from the 'promo2_start_date'
clean_train['promo2_start_month'] = clean_train['promo2_start_date'].dt.month

# Display the first few rows to verify
print(clean_train[['promo2sinceweek', 'promo2_start_date', 'promo2_start_month']].head())

   promo2sinceweek promo2_start_date  promo2_start_month
0              NaN               NaT                 NaN
1             13.0        2010-03-29                 3.0
2             14.0        2011-04-04                 4.0
3              NaN               NaT                 NaN
4              NaN               NaT                 NaN


In [8]:
# Convert 'promo2sinceweek' to a date only if both 'promo2sinceweek' and 'promo2sinceyear' are not NaN
clean_test['promo2_start_date'] = clean_test.apply(
    lambda row: pd.to_datetime(f"{int(row['promo2sinceyear'])}{int(row['promo2sinceweek']):02}1", format='%Y%W%w', errors='coerce')
    if pd.notna(row['promo2sinceweek']) and pd.notna(row['promo2sinceyear'])
    else pd.NaT,
    axis=1
)

# Extract the month from the 'promo2_start_date'
clean_test['promo2_start_month'] = clean_test['promo2_start_date'].dt.month

# Display the first few rows to verify
print(clean_test[['promo2sinceweek', 'promo2_start_date', 'promo2_start_month']].head())

   promo2sinceweek promo2_start_date  promo2_start_month
0              NaN               NaT                 NaN
1             14.0        2011-04-04                 4.0
2              NaN               NaT                 NaN
3              NaN               NaT                 NaN
4              NaN               NaT                 NaN


In [9]:
# Extract features from the 'promo2_start_date' column
clean_train['promo2_start_year'] = clean_train['promo2_start_date'].dt.year
clean_train['promo2_start_month'] = clean_train['promo2_start_date'].dt.month
clean_train['promo2_start_day'] = clean_train['promo2_start_date'].dt.day
clean_train['promo2_start_day_of_week'] = clean_train['promo2_start_date'].dt.dayofweek

clean_test['promo2_start_year'] = clean_test['promo2_start_date'].dt.year
clean_test['promo2_start_month'] = clean_test['promo2_start_date'].dt.month
clean_test['promo2_start_day'] = clean_test['promo2_start_date'].dt.day
clean_test['promo2_start_day_of_week'] = clean_test['promo2_start_date'].dt.dayofweek

# Display the first few rows to verify
print(clean_train[['promo2_start_date', 'promo2_start_year', 'promo2_start_month', 'promo2_start_day', 'promo2_start_day_of_week']].head())
print(clean_test[['promo2_start_date', 'promo2_start_year', 'promo2_start_month', 'promo2_start_day', 'promo2_start_day_of_week']].head())

  promo2_start_date  promo2_start_year  promo2_start_month  promo2_start_day  \
0               NaT                NaN                 NaN               NaN   
1        2010-03-29             2010.0                 3.0              29.0   
2        2011-04-04             2011.0                 4.0               4.0   
3               NaT                NaN                 NaN               NaN   
4               NaT                NaN                 NaN               NaN   

   promo2_start_day_of_week  
0                       NaN  
1                       0.0  
2                       0.0  
3                       NaN  
4                       NaN  
  promo2_start_date  promo2_start_year  promo2_start_month  promo2_start_day  \
0               NaT                NaN                 NaN               NaN   
1        2011-04-04             2011.0                 4.0               4.0   
2               NaT                NaN                 NaN               NaN   
3               NaT

In [10]:
clean_train.head()

,store,dayofweek,date,sales,customers,open,promo,stateholiday,schoolholiday,competitiondistance,...,storetype_d,assortment_b,assortment_c,"promointerval_Jan,Apr,Jul,Oct","promointerval_Mar,Jun,Sept,Dec",promo2_start_date,promo2_start_month,promo2_start_year,promo2_start_day,promo2_start_day_of_week
0,1,5,2015-07-31,5263,555,1,1,0,1,1270.0,...,False,False,False,False,False,NaT,NaN,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,570.0,...,False,False,False,True,False,2010-03-29,3.0,2010.0,29.0,0.0
2,3,5,2015-07-31,8314,821,1,1,0,1,14130.0,...,False,False,False,True,False,2011-04-04,4.0,2011.0,4.0,0.0
3,4,5,2015-07-31,13995,1498,1,1,0,1,620.0,...,False,False,True,False,False,NaT,NaN,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,29910.0,...,False,False,False,False,False,NaT,NaN,NaN,NaN,NaN


In [11]:
# Extract additional date features
clean_train['quarter'] = clean_train['date'].dt.quarter
clean_train['week_of_year'] = clean_train['date'].dt.isocalendar().week
clean_train['is_weekend'] = clean_train['date'].dt.dayofweek >= 5

clean_test['quarter'] = clean_test['date'].dt.quarter
clean_test['week_of_year'] = clean_test['date'].dt.isocalendar().week
clean_test['is_weekend'] = clean_test['date'].dt.dayofweek >= 5

In [12]:
clean_train.head()

,store,dayofweek,date,sales,customers,open,promo,stateholiday,schoolholiday,competitiondistance,...,"promointerval_Jan,Apr,Jul,Oct","promointerval_Mar,Jun,Sept,Dec",promo2_start_date,promo2_start_month,promo2_start_year,promo2_start_day,promo2_start_day_of_week,quarter,week_of_year,is_weekend
0,1,5,2015-07-31,5263,555,1,1,0,1,1270.0,...,False,False,NaT,NaN,NaN,NaN,NaN,3,31,False
1,2,5,2015-07-31,6064,625,1,1,0,1,570.0,...,True,False,2010-03-29,3.0,2010.0,29.0,0.0,3,31,False
2,3,5,2015-07-31,8314,821,1,1,0,1,14130.0,...,True,False,2011-04-04,4.0,2011.0,4.0,0.0,3,31,False
3,4,5,2015-07-31,13995,1498,1,1,0,1,620.0,...,False,False,NaT,NaN,NaN,NaN,NaN,3,31,False
4,5,5,2015-07-31,4822,559,1,1,0,1,29910.0,...,False,False,NaT,NaN,NaN,NaN,NaN,3,31,False


In [13]:
clean_test.head()

,id,store,dayofweek,date,open,promo,stateholiday,schoolholiday,competitiondistance,competitionopensincemonth,...,"promointerval_Jan,Apr,Jul,Oct","promointerval_Mar,Jun,Sept,Dec",promo2_start_date,promo2_start_month,promo2_start_year,promo2_start_day,promo2_start_day_of_week,quarter,week_of_year,is_weekend
0,1,1,4,2015-09-17,1.0,1,0,0,1270.0,9.0,...,False,False,NaT,NaN,NaN,NaN,NaN,3,38,False
1,2,3,4,2015-09-17,1.0,1,0,0,14130.0,12.0,...,True,False,2011-04-04,4.0,2011.0,4.0,0.0,3,38,False
2,3,7,4,2015-09-17,1.0,1,0,0,24000.0,4.0,...,False,False,NaT,NaN,NaN,NaN,NaN,3,38,False
3,4,8,4,2015-09-17,1.0,1,0,0,7520.0,10.0,...,False,False,NaT,NaN,NaN,NaN,NaN,3,38,False
4,5,9,4,2015-09-17,1.0,1,0,0,2030.0,8.0,...,False,False,NaT,NaN,NaN,NaN,NaN,3,38,False


In [14]:
#return the data with feature eng to csv
clean_train.to_csv('train_eng.csv', index=False)
clean_test.to_csv('test_eng.csv', index=False)